In [ ]:
# =========================================
# MIMIC-IV: AKI causal inference with Notes (LLM hook, single-file)
# =========================================
from __future__ import annotations

# Imports
import os, re, json, time, math, datetime as dt
import numpy as np
import pandas as pd
from google.colab import drive
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.linear_model import LogisticRegression
import gc

In [ ]:
try:
    from lifelines import CoxPHFitter
except Exception:
    CoxPHFitter = None

In [ ]:
!pip install python-dotenv -q
from dotenv import load_dotenv
from __future__ import annotations
from openai import OpenAI

In [ ]:
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/'
load_dotenv(drive_path + 'Colab Notebooks/env_config/.env')

Mounted at /content/drive


True

In [ ]:
# --- Config ---
pd.set_option("display.max_rows", 8)
MIMIC_DIR = Path("/content/drive/MyDrive/data/mimiciv/3.1/")
HOSP = MIMIC_DIR / "hosp"
NOTE = MIMIC_DIR / "note"
ICU  = MIMIC_DIR / "icu"
OUTD = MIMIC_DIR / "results_ci"; OUTD.mkdir(parents=True, exist_ok=True)
# 큰 csv.gz는 청크로
READ_KW = dict(dtype_backend="pyarrow" , low_memory=False)

# ---------- Causal / modeling ----------
RANDOM_STATE = 7
PS_CLIP      = (1e-3, 1-1e-3)
W_TRIM       = (0.01, 0.99)
VPT_WINDOW_HOURS = 6
COX_PENALIZER = 0.1

# ---------- Regex / patterns ----------
RX_SCR_LABEL  = "creatinine"                        # simple substring (no regex)
RX_SCR_FLUID  = r"\b(?:serum|blood)\b"              # regex
RX_MGDL       = r"\bmg/dl\b"
RX_MGL        = r"\bmg/l\b"
RX_VANCO_SUB  = "vancomycin"                        # simple substring
RX_PTZ        = r"(?:piperacillin|tazobactam|zosyn)"
RX_EMERG_SUB  = "EMER"                              # admission_type contains "EMER"

# 텍스트 피처(만성=confounder 후보 / 급성=collider 위험 → *_pre로 LLM에서만 사용)
TEXT_PATTERNS_CHRONIC = {
    "f_ckd":       "chronic kidney disease / ESRD / dialysis history",
    "f_dm":        "diabetes mellitus",
    "f_hf":        "heart failure (any; HFrEF/HFpEF)",
    "f_liver":     "cirrhosis / ESLD / hepatic failure",
    "f_nephrotox": "nephrotoxic baseline meds (NSAIDs, ACEi/ARB, aminoglycosides, calcineurin inhibitors)",
}
TEXT_PATTERNS_ACUTE_PRE = {
    "f_sepsis_pre":   "sepsis present before or at index (on admission)",
    "f_shock_pre":    "shock/vasopressor need before or at index",
    "f_contrast_pre": "iodinated contrast exposure before index (e.g., pre-admission CTA)",
    "f_vent_pre":     "mechanical ventilation before or at index",
}
ALL_CONCEPTS_FOR_LLM = {**TEXT_PATTERNS_CHRONIC, **TEXT_PATTERNS_ACUTE_PRE}

# ---------- LLM Config ----------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or os.getenv("API_KEY")
LLM_MODEL = "gpt-4o-mini"
MAX_NOTE_CHARS = 15000
LLM_BATCH_SIZE = 24
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 200           # 짧은 JSON만 반환
LLM_TIMEOUT = 60
CONFOUNDERS = ["f_ckd_pre", "f_dm_pre", "f_hf_pre", "f_liver_pre", "f_nephrotox_pre"]

# ---------- LLM prompt (pre-treatment only) ----------
LLM_PROMPT_TEMPLATE = """
You are assisting a causal inference study of AKI. Exposure = vancomycin±piperacillin/tazobactam.
Your ONLY task: read a discharge note and mark **pre-treatment** (pre-admission or at presentation) risk factors.

Rules:
- Consider ONLY information existing **before or at presentation** relative to index_time = {index_time_iso}.
- DO NOT mark conditions/events clearly arising during hospitalization, hospital course, ICU interventions, inpatient treatments, or discharge meds. Those are potential colliders.
- If timing is ambiguous, be conservative and mark 0.
- Output a compact ONE-LINE JSON with 0/1 integers. No extra text.

Binary variables (confounders of interest):
- f_ckd_pre: chronic kidney disease / ESRD / chronic dialysis BEFORE presentation.
- f_dm_pre: diabetes mellitus history.
- f_hf_pre: heart failure (any phenotype), chronic.
- f_liver_pre: cirrhosis or end-stage liver disease (chronic).
- f_nephrotox_pre: chronic/home exposure to nephrotoxins (NSAIDs, ACEi/ARB, calcineurin inhibitors, chronic aminoglycosides). Ignore inpatient doses.

Return ONLY:
{{
  "f_ckd_pre": 0 or 1,
  "f_dm_pre": 0 or 1,
  "f_hf_pre": 0 or 1,
  "f_liver_pre": 0 or 1,
  "f_nephrotox_pre": 0 or 1
}}

Discharge note:
---
{note_text}
---
""".strip()

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
tmp = pd.read_csv(NOTE/"discharge_detail.csv.gz", nrows=3)
print("discharge_detail columns:", list(tmp.columns))
tmp2 = pd.read_csv(NOTE/"discharge.csv.gz", nrows=3)
print("discharge columns:", list(tmp2.columns))

discharge_detail columns: ['note_id', 'subject_id', 'field_name', 'field_value', 'field_ordinal']
discharge columns: ['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime', 'text']


In [ ]:
# ---------- Utils ----------
def _to_datetime_safe(s: pd.Series, fmt: str | None = "%Y-%m-%d %H:%M:%S") -> pd.Series:
    x = pd.to_datetime(s, errors="coerce", format=fmt)
    if x.isna().any():
        y = pd.to_datetime(s[x.isna()], errors="coerce")
        x.loc[x.isna()] = y
    return x

def _assert_has(df: pd.DataFrame, cols: list[str], name="df"):
    miss = [c for c in cols if c not in df.columns]
    if miss:
        raise ValueError(f"[{name}] missing columns: {miss}")

def _smd(x, t, w=None):
    x = np.asarray(x, float); t = np.asarray(t, int)
    if w is None: w = np.ones_like(t, float)
    if (t==1).sum()==0 or (t==0).sum()==0: return np.nan
    m1 = np.average(x[t==1], weights=w[t==1]); m0 = np.average(x[t==0], weights=w[t==0])
    v1 = np.average((x[t==1]-m1)**2, weights=w[t==1]); v0 = np.average((x[t==0]-m0)**2, weights=w[t==0])
    return (m1-m0)/np.sqrt((v1+v0)/2 + 1e-9)

def evalue_from_hr(hr, lcl, ucl):
    def _ev(x: float) -> float:
        return x + math.sqrt(max(x,0)*(max(x,0)-1.0)) if x>1 else 1.0
    return (_ev(float(hr)), _ev(float(lcl)) if float(lcl)>1 else 1.0)

In [ ]:
# =========================
# 1) Treatment cohort (V vs VPT)
# =========================
def build_cohort(hosp: Path) -> pd.DataFrame:
    use_cols = ["subject_id","hadm_id","starttime","stoptime","drug"]
    it = pd.read_csv(hosp/"prescriptions.csv.gz",
                     usecols=lambda c: c in use_cols,
                     chunksize=500_000, low_memory=False)
    v_chunks, p_chunks = [], []

    for ch in it:
        ch["starttime"] = _to_datetime_safe(ch["starttime"])
        ch["stoptime"]  = _to_datetime_safe(ch["stoptime"])
        dlow = ch["drug"].astype("string").str.lower()

        v = ch[dlow.str.contains(RX_VANCO_SUB, na=False, regex=False)]
        p = ch[dlow.str.contains(RX_PTZ,       na=False, regex=True)]

        v = v.rename(columns={"starttime":"v_start","stoptime":"v_stop"})
        p = p.rename(columns={"starttime":"p_start","stoptime":"p_stop"})
        v_chunks.append(v[["subject_id","hadm_id","v_start","v_stop"]])
        p_chunks.append(p[["subject_id","hadm_id","p_start","p_stop"]])

    vanco = pd.concat(v_chunks, ignore_index=True) if v_chunks else \
            pd.DataFrame(columns=["subject_id","hadm_id","v_start","v_stop"])
    ptz   = pd.concat(p_chunks, ignore_index=True) if p_chunks else \
            pd.DataFrame(columns=["subject_id","hadm_id","p_start","p_stop"])

    v1 = (vanco.sort_values("v_start")
                .groupby(["subject_id","hadm_id"], as_index=False)
                .agg(index_time=("v_start","first")))
    p1 = (ptz.sort_values("p_start")
              .groupby(["subject_id","hadm_id"], as_index=False)
              .agg(ptz_time=("p_start","first")))

    v1["index_time"] = _to_datetime_safe(v1["index_time"])
    p1["ptz_time"]   = _to_datetime_safe(p1["ptz_time"])

    cohort = v1.merge(p1, how="left", on=["subject_id","hadm_id"])
    cohort = cohort.dropna(subset=["index_time"]).reset_index(drop=True)

    # STRICT: ptz_time >= index_time and <= index_time + 6h
    cohort["vpt_flag"] = (
        cohort["ptz_time"].notna() &
        (cohort["ptz_time"] >= cohort["index_time"]) &
        (cohort["ptz_time"] <= cohort["index_time"] + pd.Timedelta(hours=VPT_WINDOW_HOURS))
    ).astype(int)

    for k in ("subject_id","hadm_id"):
        cohort[k] = cohort[k].astype("Int64")

    print(f"[cohort] N={len(cohort)}  VPT={int(cohort['vpt_flag'].sum())}")
    return cohort


In [ ]:
# =========================
# 2) Labs → SCr & AKI labels
# =========================
def load_scr_itemids(hosp: Path) -> list[int]:
    d_lab = pd.read_csv(hosp/"d_labitems.csv.gz", **READ_KW)
    # 1) strict: label contains 'creatinine' and fluid mentions serum/blood
    mask = (
        d_lab["label"].astype("string").str.contains(RX_SCR_LABEL, na=False, regex=False, case=False) &
        d_lab["fluid"].astype("string").str.contains(RX_SCR_FLUID, na=False, regex=True, case=False)
    )
    ids = d_lab.loc[mask, "itemid"].dropna().astype("Int64").astype(int).unique().tolist()

    # 2) fallback: if none, ignore fluid filter (dataset 변형 대응)
    if len(ids) == 0:
        mask2 = d_lab["label"].astype("string").str.contains(RX_SCR_LABEL, na=False, regex=False, case=False)
        ids = d_lab.loc[mask2, "itemid"].dropna().astype("Int64").astype(int).unique().tolist()

    print(f"[scr ids] K={len(ids)}  sample: {ids[:3]}")
    return ids

def load_scr_timeseries(hosp: Path, cohort: pd.DataFrame, scr_ids: list[int]) -> pd.DataFrame:
    if len(scr_ids)==0:
        raise RuntimeError("No SCr itemids found. Check d_labitems.csv.gz filters.")

    hadm_set = set(cohort["hadm_id"].dropna().astype(int).unique().tolist())
    keep = ["subject_id","hadm_id","itemid","charttime","valuenum","valueuom"]

    it = pd.read_csv(hosp/"labevents.csv.gz",
                     usecols=lambda c: c in keep,
                     chunksize=1_000_000, low_memory=False)
    chunks=[]
    for ch in it:
        ch = ch[ch["itemid"].isin(scr_ids)]
        ch = ch[ch["hadm_id"].isin(hadm_set)]
        if len(ch)==0:
            continue
        ch["charttime"] = _to_datetime_safe(ch["charttime"])

        u = ch["valueuom"].astype("string").str.lower()
        ch["scr_mgdl"] = ch["valuenum"]

        m_mgl  = u.str.contains(RX_MGL,  na=False, regex=True)
        m_mgdl = u.str.contains(RX_MGDL, na=False, regex=True)

        # mg/L → mg/dL
        ch.loc[m_mgl, "scr_mgdl"] = ch.loc[m_mgl, "valuenum"] / 10.0

        # 허용 단위만 유지
        ch = ch[m_mgdl | m_mgl]
        if len(ch):
            chunks.append(ch[["subject_id","hadm_id","itemid","charttime","scr_mgdl"]])

    df = pd.concat(chunks, ignore_index=True) if chunks else \
         pd.DataFrame(columns=["subject_id","hadm_id","itemid","charttime","scr_mgdl"])

    df = df.merge(cohort[["subject_id","hadm_id","index_time","vpt_flag"]],
                  on=["subject_id","hadm_id"], how="inner")

    if len(df)==0:
        print("[scr ts] rows=0 (no overlapping SCr rows for cohort)")
        return df

    df["index_time"] = _to_datetime_safe(df["index_time"])
    df = df.dropna(subset=["charttime","index_time"]).reset_index(drop=True)
    df["dt"] = (df["charttime"] - df["index_time"]).dt.total_seconds()/3600.0
    df = df.sort_values(["subject_id","hadm_id","charttime"]).reset_index(drop=True)
    for k in ("subject_id","hadm_id"):
        df[k] = df[k].astype("Int64")
    print(f"[scr ts] rows={len(df)}")
    return df

def label_aki(df: pd.DataFrame, cohort: pd.DataFrame) -> pd.DataFrame:
    if len(df)==0:
        out = cohort.copy()
        out["baseline"] = np.nan
        out["aki48"] = 0; out["aki7x"] = 0; out["aki"] = 0
        print(f"[aki] rate=0.000  N={len(out)} (no SCr rows)")
        return out

    base = (df[(df["dt"]>=-24) & (df["dt"]<=0)]
              .groupby(["subject_id","hadm_id"], as_index=False)
              .agg(baseline=("scr_mgdl","median")))
    out = cohort.merge(base, on=["subject_id","hadm_id"], how="left")

    if out["baseline"].isna().any():
        first24 = (df[(df["dt"]>=0) & (df["dt"]<=24)]
                     .sort_values("charttime")
                     .groupby(["subject_id","hadm_id"], as_index=False)
                     .agg(first24=("scr_mgdl","first")))
        out = out.merge(first24, on=["subject_id","hadm_id"], how="left")
        out["baseline"] = out["baseline"].fillna(out["first24"])

    p48 = (df[(df["dt"]>0) & (df["dt"]<=48)]
             .groupby(["subject_id","hadm_id"], as_index=False)
             .agg(max48=("scr_mgdl","max")))
    p7d = (df[(df["dt"]>0) & (df["dt"]<=24*7)]
             .groupby(["subject_id","hadm_id"], as_index=False)
             .agg(max7=("scr_mgdl","max")))

    out = out.merge(p48, on=["subject_id","hadm_id"], how="left") \
             .merge(p7d, on=["subject_id","hadm_id"], how="left")

    out["aki48"] = (out["max48"] >= (out["baseline"] + 0.3)).astype(int)
    out["aki7x"] = (out["max7"]  >= (1.5 * out["baseline"])).astype(int)
    out["aki"]   = ((out["aki48"]==1) | (out["aki7x"]==1)).astype(int)

    print(f"[aki] rate={out['aki'].mean():.3f}  N={len(out)}")
    return out

In [ ]:
# =========================
# 3) Time-to-event window (≤7d)
# =========================
def build_event_times(df_scr: pd.DataFrame, out: pd.DataFrame, hosp: Path) -> pd.DataFrame:
    adm = pd.read_csv(hosp/"admissions.csv.gz",
                      usecols=["subject_id","hadm_id","admittime","dischtime"],
                      parse_dates=["admittime","dischtime"], **READ_KW)
    adm["admittime"] = _to_datetime_safe(adm["admittime"])
    adm["dischtime"] = _to_datetime_safe(adm["dischtime"])

    if len(df_scr)==0:
        evt = out[["subject_id","hadm_id","index_time","vpt_flag"]].copy()
        evt["event_time_48"] = pd.NaT
        evt["event_time_7"]  = pd.NaT
        evt["event_time"]    = pd.NaT
    else:
        tmp48 = (df_scr[(df_scr["dt"]>0) & (df_scr["dt"]<=48)]
                 .merge(out[["subject_id","hadm_id","baseline"]],
                        on=["subject_id","hadm_id"], how="left"))
        hit48 = tmp48[tmp48["scr_mgdl"] >= (tmp48["baseline"] + 0.3)]
        ev48 = (hit48.sort_values("charttime")
                     .groupby(["subject_id","hadm_id"], as_index=False)
                     .first()[["subject_id","hadm_id","charttime"]]
                     .rename(columns={"charttime":"event_time_48"}))

        tmp7 = (df_scr[(df_scr["dt"]>0) & (df_scr["dt"]<=24*7)]
                .merge(out[["subject_id","hadm_id","baseline"]],
                       on=["subject_id","hadm_id"], how="left"))
        hit7 = tmp7[tmp7["scr_mgdl"] >= (1.5 * tmp7["baseline"])]
        ev7 = (hit7.sort_values("charttime")
                   .groupby(["subject_id","hadm_id"], as_index=False)
                   .first()[["subject_id","hadm_id","charttime"]]
                   .rename(columns={"charttime":"event_time_7"}))

        evt = (out[["subject_id","hadm_id","index_time","vpt_flag"]]
               .merge(ev48, how="left")
               .merge(ev7,  how="left"))
        evt["event_time"] = evt[["event_time_48","event_time_7"]].min(axis=1)

    evt = evt.merge(adm, on=["subject_id","hadm_id"], how="left")
    evt["censor_limit"] = evt["index_time"] + pd.Timedelta(days=7)
    evt["censor_time"]  = evt[["dischtime","censor_limit"]].min(axis=1)

    evt = evt.dropna(subset=["index_time","censor_time"]).reset_index(drop=True)
    evt["event_observed"] = (evt["event_time"].notna()) & (evt["event_time"] <= evt["censor_time"])
    evt["duration_days"] = (
        np.where(
            evt["event_observed"],
            (evt["event_time"] - evt["index_time"]).dt.total_seconds(),
            (evt["censor_time"] - evt["index_time"]).dt.total_seconds()
        ) / 86400.0
    ).clip(min=0)

    for k in ("subject_id","hadm_id"):
        evt[k] = evt[k].astype("Int64")

    print(f"[tte] rows={len(evt)} events={int(evt['event_observed'].sum())}")
    return evt

In [ ]:
# =========================
# 4) Notes → LLM hook (or regex fallback)
# =========================
def read_discharge_safe(note_dir: Path) -> pd.DataFrame:
    raw = pd.read_csv(note_dir/"discharge.csv.gz", **READ_KW)
    # 컬럼명이 다른 경우를 고려하여 안전 인식
    # 기본 세트
    cand_sid  = [c for c in raw.columns if c.lower()=="subject_id"]
    cand_hadm = [c for c in raw.columns if c.lower()=="hadm_id"]
    cand_text = [c for c in raw.columns if c.lower() in ("text","note_text")]
    if not (cand_sid and cand_hadm and cand_text):
        raise ValueError(f"[discharge.csv] required columns not found. Have={list(raw.columns)}")

    df = raw[[cand_sid[0], cand_hadm[0], cand_text[0]]].copy()
    df.columns = ["subject_id","hadm_id","text"]
    df["text"] = df["text"].astype("string[python]").fillna("")
    for k in ("subject_id","hadm_id"): df[k] = df[k].astype("Int64")
    return df

def _extract_excerpt(text: str, max_chars=MAX_NOTE_CHARS) -> str:
    return (text[:max_chars] if isinstance(text,str) else "")

def _openai_json_call(prompt: str) -> dict:
    if OpenAI is None or not OPENAI_API_KEY:
        raise RuntimeError("OpenAI not available (package or API key).")
    client = OpenAI(api_key=OPENAI_API_KEY)
    resp = client.chat.completions.create(
        model=LLM_MODEL,
        response_format={"type":"json_object"},
        messages=[
            {"role":"system","content":"Return strict JSON only."},
            {"role":"user","content":prompt}
        ],
        temperature=LLM_TEMPERATURE,
    )
    return json.loads(resp.choices[0].message.content)

def make_llm_extractor_openai(index_time_map: dict, cache_dir: Path|None=None):
    if cache_dir: cache_dir.mkdir(parents=True, exist_ok=True)

    def llm_fn(df_notes: pd.DataFrame) -> pd.DataFrame:
        rows=[]
        for (sid, hid), g in df_notes.groupby(["subject_id","hadm_id"]):
            text = "\n".join(g["text"].astype(str).tolist())
            excerpt = _extract_excerpt(text)
            # index_time (UTC ISO) — 없는 경우 보수적으로 UNKNOWN 사용 (prompt 보수적 규칙이 0을 유도)
            it = index_time_map.get((int(sid), int(hid)))
            it_iso = pd.to_datetime(it).tz_localize("UTC").isoformat() if pd.notna(it) else "UNKNOWN"
            prompt = LLM_PROMPT_TEMPLATE.format(index_time_iso=it_iso, note_text=excerpt)
            try:
                raw = _openai_json_call(prompt)
            except Exception as e:
                print(f"[warn] LLM fail hadm={hid}: {e} -> zero-fill")
                raw = {k:0 for k in CONFOUNDERS}
            row = {"subject_id":int(sid), "hadm_id":int(hid)}
            row.update({k:int(raw.get(k,0)) for k in CONFOUNDERS})
            rows.append(row)
        return pd.DataFrame(rows)

    return llm_fn

# Regex fallback (chronic only) → collider 회피
TEXT_PATTERNS_CHRONIC = {
    "f_ckd_pre":       r"\b(?:ckd|chronic kidney disease|chronic renal (?:failure|insufficiency)|esrd|end[- ]?stage renal disease|hemodialysis|peritoneal dialysis)\b",
    "f_dm_pre":        r"\b(?:diabetes|dm|diabetic)\b",
    "f_hf_pre":        r"\b(?:heart failure|chf|hfref|hfp ef|hfpef|hf p?ef)\b",
    "f_liver_pre":     r"\b(?:cirrhosis|end[- ]?stage liver disease|esld|hepatic failure)\b",
    "f_nephrotox_pre": r"\b(?:nsaid|ibuprofen|naproxen|ketorolac|ace inhibitor|acei|arb|gentamicin|amikacin|tobramycin|cyclosporine|tacrolimus)\b",
}
def regex_features_from_text(df_notes: pd.DataFrame) -> pd.DataFrame:
    out = df_notes[["subject_id","hadm_id"]].drop_duplicates().copy()
    for name, pat in TEXT_PATTERNS_CHRONIC.items():
        rx = re.compile(pat, re.I)
        hit = df_notes.groupby(["subject_id","hadm_id"])["text"] \
                      .apply(lambda s: s.str.contains(rx, na=False).any()) \
                      .astype(int).reset_index(name=name)
        out = out.merge(hit, on=["subject_id","hadm_id"], how="left")
    for c in out.columns:
        if c not in ("subject_id","hadm_id"):
            out[c] = out[c].fillna(0).astype(int)
    return out

In [ ]:
# =========================
# 5) Covariates + PS + Survival models
# =========================
def build_covariates(hosp: Path, out: pd.DataFrame, note_feat: pd.DataFrame) -> pd.DataFrame:
    adm = pd.read_csv(hosp/"admissions.csv.gz",
                      usecols=["subject_id","hadm_id","admittime","dischtime","admission_type"],
                      parse_dates=["admittime","dischtime"], **READ_KW)
    pat = pd.read_csv(hosp/"patients.csv.gz",
                      usecols=["subject_id","gender","anchor_age"], **READ_KW)
    adm["admittime"] = _to_datetime_safe(adm["admittime"])
    adm["dischtime"] = _to_datetime_safe(adm["dischtime"])

    dfc = (out.merge(adm, on=["subject_id","hadm_id"], how="left")
              .merge(pat, on="subject_id", how="left")
              .merge(note_feat, on=["subject_id","hadm_id"], how="left"))
    dfc["age"]      = pd.to_numeric(dfc["anchor_age"], errors="coerce")
    dfc["sexM"]     = (dfc["gender"] == "M").astype(int)
    dfc["is_emerg"] = dfc["admission_type"].astype("string") \
                        .str.contains(RX_EMERG_SUB, na=False, regex=False).astype(int)

    for c in CONFOUNDERS:
        if c in dfc.columns: dfc[c] = dfc[c].fillna(0).astype(int)
    return dfc

def fit_ps_and_sw(dfc: pd.DataFrame, covs: list[str],
                  ps_clip=PS_CLIP, w_trim=W_TRIM):
    covs = [c for c in dict.fromkeys(covs) if c in dfc.columns]
    if not covs:
        raise ValueError("No covariates provided to fit_ps_and_sw.")
    X = dfc[covs].copy()
    for c in covs: X[c] = pd.to_numeric(X[c], errors="coerce")
    X = X.fillna(X.median(numeric_only=True))

    T = dfc["vpt_flag"].astype(int)
    ps_model = LogisticRegression(max_iter=400, solver="lbfgs", random_state=RANDOM_STATE)
    ps_model.fit(X, T)
    ps = np.clip(ps_model.predict_proba(X)[:,1], *ps_clip)

    p_t = float(T.mean())
    sw = np.where(T==1, p_t/ps, (1-p_t)/(1-ps))
    lo, hi = np.quantile(sw, w_trim); sw = np.clip(sw, lo, hi)

    ESS = float((sw.sum()**2)/(sw**2).sum())
    return X, ps, sw, ESS

def cox_results_ipw_and_dr(dfc: pd.DataFrame, sw: np.ndarray, covs: list[str], penalizer=COX_PENALIZER):
    if CoxPHFitter is None:
        raise RuntimeError("lifelines is not installed.")
    d = pd.DataFrame({
        "time":  pd.to_numeric(dfc["duration_days"], errors="coerce"),
        "event": dfc["event_observed"].astype(int),
        "treat": dfc["vpt_flag"].astype(int),
        "sw":    sw
    }).dropna(subset=["time","event","treat","sw"])

    if d["event"].sum() == 0:
        raise RuntimeError("No events in TTE window; cannot fit Cox model.")

    cph_w = CoxPHFitter(penalizer=penalizer)
    cph_w.fit(d, duration_col="time", event_col="event", weights_col="sw", robust=True)
    iptw_HR  = float(np.exp(cph_w.params_["treat"]))
    iptw_LCL, iptw_UCL = np.exp(cph_w.confidence_intervals_.loc["treat"].values)

    X = dfc[covs].copy()
    for c in covs: X[c] = pd.to_numeric(X[c], errors="coerce")
    X = X.fillna(X.median(numeric_only=True))
    d2 = pd.concat([d.reset_index(drop=True), X.reset_index(drop=True)], axis=1)

    cph_dr = CoxPHFitter(penalizer=penalizer)
    cph_dr.fit(d2, duration_col="time", event_col="event", weights_col="sw", robust=True)
    dr_HR  = float(np.exp(cph_dr.params_["treat"]))
    dr_LCL, dr_UCL = np.exp(cph_dr.confidence_intervals_.loc["treat"].values)

    return (iptw_HR, iptw_LCL, iptw_UCL), (dr_HR, dr_LCL, dr_UCL)

def evaluate_covset(dfc: pd.DataFrame, covs: list[str], label="BASE"):
    covs = [c for c in dict.fromkeys(covs) if c in dfc.columns]
    if not covs: raise ValueError("No covariates for evaluate_covset.")

    X, ps, sw, ESS = fit_ps_and_sw(dfc, covs)

    T = dfc["vpt_flag"].astype(int).values
    smd_b = [abs(_smd(X[c].values, T, None)) for c in X.columns]
    smd_a = [abs(_smd(X[c].values, T, sw))  for c in X.columns]
    meanSMD_b = float(np.nanmean(smd_b)); meanSMD_a = float(np.nanmean(smd_a))

    # PS overlap (KS)
    try:
        from scipy.stats import ks_2samp
        KS = float(ks_2samp(ps[T==1], ps[T==0]).statistic)
    except Exception:
        grid = np.linspace(0,1,200)
        c1 = np.searchsorted(np.sort(ps[T==1]), grid, side="right")/max(1,(T==1).sum())
        c0 = np.searchsorted(np.sort(ps[T==0]), grid, side="right")/max(1,(T==0).sum())
        KS = float(np.max(np.abs(c1-c0)))

    (hr_i,l_i,u_i),(hr_d,l_d,u_d) = cox_results_ipw_and_dr(dfc, sw, covs)
    E_point, E_CI = evalue_from_hr(hr_i, l_i, u_i)

    return {
        "covset":label, "k_covs":len(covs),
        "mean_abs_SMD_before":meanSMD_b, "mean_abs_SMD_after":meanSMD_a,
        "ESS":ESS, "KS_PS":KS,
        "IPTW_HR":hr_i, "IPTW_LCL":l_i, "IPTW_UCL":u_i, "IPTW_CI_width":u_i-l_i,
        "DR_HR":hr_d,   "DR_LCL":l_d,  "DR_UCL":u_d,  "DR_CI_width":u_d-l_d,
        "Evalue_point":E_point, "Evalue_CI":E_CI
    }

In [ ]:
# =========================
# 6) Main end-to-end
# =========================
def run_pipeline_with_llm(HOSP: Path, NOTE: Path, use_llm: bool=True):
    # 1) Cohort
    cohort = build_cohort(HOSP)

    # 2) Labs → AKI
    scr_ids = load_scr_itemids(HOSP)
    df_scr  = load_scr_timeseries(HOSP, cohort, scr_ids)
    out     = label_aki(df_scr, cohort)

    # 3) TTE
    evt = build_event_times(df_scr, out, HOSP)

    # 4) Notes
    notes = read_discharge_safe(NOTE)

    # LLM index_time map (for prompt)
    idx_map = {(int(r.subject_id), int(r.hadm_id)): r.index_time
               for _, r in out[["subject_id","hadm_id","index_time"]].dropna().iterrows()}

    # LLM or regex fallback
    if use_llm:
        try:
            llm_fn = make_llm_extractor_openai(index_time_map=idx_map, cache_dir=OUTD/"llm_cache")
            note_feat = llm_fn(notes)
        except Exception as e:
            print(f"[warn] LLM extractor failed: {e} -> regex(chronic) fallback")
            note_feat = regex_features_from_text(notes)
    else:
        note_feat = regex_features_from_text(notes)

    # 5) Covariates
    dfc = build_covariates(HOSP, out, note_feat)
    dfc = dfc.merge(evt[["subject_id","hadm_id","duration_days","event_observed"]],
                    on=["subject_id","hadm_id"], how="left")
    dfc = dfc.dropna(subset=["vpt_flag","aki","duration_days","event_observed"]).reset_index(drop=True)

    print(f"[analytic] N={len(dfc)}  events={int(dfc['event_observed'].sum())}  treated={int(dfc['vpt_flag'].sum())}")

    # 6) Evaluate BASE vs BASE+LLM
    base_covs = ["age","sexM","is_emerg","baseline"]
    llm_covs  = base_covs + [c for c in CONFOUNDERS if c in dfc.columns]

    m_base = evaluate_covset(dfc, base_covs, "BASE")
    m_llm  = evaluate_covset(dfc, llm_covs,  "BASE+LLM")

    perf = pd.DataFrame([m_base, m_llm])
    perf.to_csv(OUTD/"perf_base_vs_llm.csv", index=False)

    print("\n=== Performance (BASE vs BASE+LLM) ===")
    cols_show = [
        "covset","k_covs",
        "mean_abs_SMD_before","mean_abs_SMD_after",
        "ESS","KS_PS",
        "IPTW_HR","IPTW_LCL","IPTW_UCL",
        "DR_HR","DR_LCL","DR_UCL",
        "Evalue_point","Evalue_CI"
    ]
    print(perf[cols_show])
    print(f"[saved] {OUTD/'perf_base_vs_llm.csv'}")

    return dfc, perf

In [ ]:
# =========================
# RUN
# =========================
if __name__ == "__main__":
    use_llm = True  # LLM 사용을 끄려면 False
    try:
        dfc, perf = run_pipeline_with_llm(HOSP, NOTE, use_llm=use_llm)
    except Exception as e:
        print("[FATAL]", e)

[cohort] N=90327  VPT=7822
[scr ids] K=3  sample: [50912, 52024, 52546]
[scr ts] rows=1172668
[aki] rate=0.175  N=90327
[tte] rows=90327 events=15811


KeyboardInterrupt: 

In [ ]:
# =========================
# RUN
# =========================
if __name__ == "__main__":
    use_llm = True  # LLM 사용을 끄려면 False
    try:
        dfc, perf = run_pipeline_with_llm(HOSP, NOTE, use_llm=use_llm)
    except Exception as e:
        print("[FATAL]", e)

[cohort] N=90327  VPT=7822
[scr ids] K=3  sample: [50912, 52024, 52546]
[scr ts] rows=1172668
[aki] rate=0.175  N=90327
[tte] rows=90327 events=15811
[warn] LLM fail hadm=22595853: name 'LLM_TEMPERATURE' is not defined -> zero-fill
[warn] LLM extractor failed: name 'CONFOUNDERS' is not defined -> regex(chronic) fallback
[FATAL] name 'CONFOUNDERS' is not defined


In [ ]:
!jupyter nbconvert --to notebook --execute --inplace --allow-errors *.ipynb
